# PS 88 - Lab 13 - DiD and Fixed Effects

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline

## Part 1: Propaganda News

Let's explore a simulated example of media persuasion with more than two time periods and more than two people. 

Suppose the president of a semi-democratic country tries to increase his grip on power by helping create Propaganda News Channel (PNC). The president was gradually increasing his popularity over time, and PNC was introduced 5 years into his term. The managers of the station also generally targeted regions with more people who support the president to get higher viewership. 

As we will see, this creates multiple sources of selection bias in trying to estimate the causal effect of PNC, but the one-two punch of two-way fixed effects will knock all of them out.

We will study some hypothetical survey data, collected over 10 years (2000-2009). Each respondent lives in one of four regions, which we label 1-4. Citizens in "higher" regions tend to like the president more. Our key outcome variable will be the approval rating of the president, which ranges from 0 to 100.

PNC is available in regions 3 and 4 starting in 2006.

The simulation generates the potential outcomes of approval without access to PNC, and then adds the true causal effect (a 5 point approval boost) to those where PNC is available. 

In [ ]:
np.random.seed(88)
# True causal effect
k=5
# Generating variables
yr = np.random.randint(2000,2010, 100000)
region = np.random.randint(1,5, size=100000)
approve0 = 60*np.random.rand(100000) + 2*np.sqrt(yr - 2000) +2*region**2
avail = 1*(region > 2)*(yr > 2005)
approve = np.where(avail==1, approve0 + k, approve0)
pncdata = pd.DataFrame({"Year":yr, "Region": region, "Available":avail, "Approve":approve})
#pncdata = pd.DataFrame({"Year":yr, "Region": region, "Available":avail, "Watch":watch, "Approve":approve})
pncdata

To check the simulation did what we want and get some practice with exploring the data, let's look at a crosstab (like a pivot table) that counts how many respondents there are by `Year` and `Available`. We can do this with `pd.crosstab`, where we input as arguments two columns, and the output is a count of how many cases take on each combination of values of the two columns:

In [ ]:
pd.crosstab(pncdata['Year'],pncdata['Available'])

**Question 1.1 Create a crosstab that counts how many respondents had PNC available by region**

In [ ]:
# Code for 1.1

We can plot the overall approval over time with `sns.lineplot`.

In [ ]:
sns.lineplot(x='Year', y='Approve', data=pncdata, ci=None)

Clearly this is trending up. Since the station was rolled out later in the time window, this might create confounding as those who have access to PNC will be in the later years.

We can also compute the average approval by region using the `groupby` function`. 

In [ ]:
pncdata.groupby("Region")['Approve'].mean()

There are big differences here, with approval much higher in the regions that got PNC (3 and 4). But this might be because the president is just more popular there in general.

What is the average approval for people with access to PNC vs. not? To facilitate later comparisons, let's use `smf.ols` to run a linear regression to answer this.

In [ ]:
smf.ols('Approve ~ Available', data=pncdata).fit().summary()

The available coefficient tells us that approval of the president is much higher among those with access to PNC, and this difference is much bigger than the real causal effect of 5.

Now let's add some "region fixed effects". Since our region variable is numeric, there is nothing technically wrong with running the following regression:

In [ ]:
smf.ols('Approve ~ Available + Region', data=pncdata).fit().summary()

This tells us that as the region goes up by "one unit", approval goes up by about 10%. But going up by one unit could correspond to going from region 1 to 2, 2 to 3, or 3 to 4. And there is no reason to think that all of these changes similar. If we want to add fixed effects, we need to add a `C()` around the Region variable in our regression formula.

In [ ]:
smf.ols('Approve ~ Available + C(Region)', data=pncdata).fit().summary()

The output is a bit goofy, but we can see that it is creating dummy variabels for regions 2, 3, and 4 (not sure what the T is about here). Since region 1 doesn't have a dummy variable, it is the *reference category*. The coefficient on `C(Region)[T.2]` tells us how much higher the approval is in region 2 vs region 1, on average, keeping fixed all other variables. The coefficients on the other regions are how much higher (or lower) the approval is compared to the referent category of region 1. 

Note the coefficient on `Available` goes down when we add the region fixed effects. This means that a lot of the difference in average approval among those with access to PNC vs not was driven by the fact that it was only available in regions that already liked the president.

**Question 1.2. Fit and summarize a regression predicting Approval with `Available` and the year fixed effects (but no region fixed effects). How does the estimate of the effect of PNC compare to the real causal effect of 5?**

In [ ]:
# Code for 1.2

*Words for 1.2*

**Question 1.3. Now run a regression predicting Approval with `Available`, the year fixed effects, and the region fixed effects. Does this produce an estimate close to the real casual effect of 5?**

In [ ]:
# Code for 1.3

*Words for 1.3*

This method can also work even if there isn't just a single time period where one group gets the treatment. 

Here is some alternative simulated data about ("Lame Propaganda News Channel") in a different country, which contains bad propaganda which actually lowers approval by one unit.  LPNC is "rolled out" by region, first going to region 4, then 3, then 2, then 1. 

For this simulation, we will assume that the real causal effect of LPNC availability is to *decrease* approval by one unit.

In [ ]:
np.random.seed(8888)
avail2 = 1*(region + yr > 2006)
approve2 = np.where(avail2==1, approve0 -1, approve0)
lpncdata = pd.DataFrame({"Year":yr, "Region": region, "Available":avail2, "Approve":approve2})
lpncdata

**Question 1.4. Use `sns.lineplot` to show what proportion of citizens have access to LPNC by year.**

In [ ]:
# Code for 1.5

**Question  1.5. Show that a regression with only year fixed effects does not give a good estimate of the causal effect of LPNC availability on presidential approval, but a regression with both year and region fixed effects gives a good estimate of the causal effect.**

In [ ]:
# Code for 1.5

## Part 2: Bringing it back full circle

Let's revisit an example from week 1: did the expiration of the Federal Assault Weapon ban lead to more homicides in areas of Mexico which are close to states in the US where the expiration made guns more accessible? In week 1 we just replicated a graph, now let's dig into the full data file from the <a href="https://www.jstor.org/stable/43654914">paper</a> by Dube, Dube, and Garcia-Ponce.

The analysis we will do is related to Table 2 in their paper, though they use a somewhat different regression method (the results are similar).

In [ ]:
ddg = pd.read_stata("cross_border_main.dta")
ddg

Each row is corresponds to a year in a city (Municipio), for 2002-2006. The main dependent variables are the number of homicides (`homicides`) and gun-related homicides (`homdguns`) in that city-year. The "treatment" here is being in a city not bordering CA after 2004. 

First, we follow the paper in restricting analysis to cities close to the border.

In [ ]:
ddg = ddg[ddg['border']==1].reset_index()

I'd like to give our main variables some more intuitive names. We are going to rename the variable indicating the city is close to a non-CA state as `non_ca`, and the indicator for being in one of these cities after 2004 as `Expire`. We can do this with the `rename` function (we'll talk about this more next week).

In [ ]:
ddg.rename(columns={"NCAseg18xpost":"Expire", "NCAseg18":"non_ca"}, inplace=True)

Let's do a crosstab of the year and whether the ban expired:

In [ ]:
pd.crosstab(ddg['year'], ddg['Expire'])

Notice that the "year" also includes a "day" and "month", which is just a function of how dates are getting passed from csv format to the notebook. The following line of code drops the 01-01.

In [ ]:
ddg['year'] = pd.DatetimeIndex(ddg['year']).year
ddg

One notable thing here is that we don't have a lot of cities close to CA: we can see this because there are just 3 observations in 2005 and 2006 where the ban didn't expire.

**Question 2.1. Use `smf.ols` to run a regression with the count of homicides (`homicide`) and as the dependent variable and `Expire` as the independent variable. Why might this not be a good estimate of the effect of the expiration of the ban?**

In [ ]:
# Code for 2.1

*Words for 2.1*

**Question 2.2. Use `smf.ols` to run a regression with the number of homicides as the dependent variable, and `Expire` and city fixed effects as dependent variables. (Hint: the `munname` variable is a string corresponding to each city, so we don't need to create a new to add fixed effects.)**

In [ ]:
# Code for 2.2

**Question 2.3. Now run the previous regression but with year fixed effects added (remember to use a `C()` to treat the year as categorical). How does this affect our estimate of the causal effect in question?**

In [ ]:
# Code for 2.3

*Words for 2.3*

**Question 2.4. Repeat this analysis with gun-related homicides (`homdguns`) as the dependent variable. What is the estimate of the causal effect of the expiration of the ban (using city and year fixed effects)?**

In [ ]:
# Code for 2.4

*Words for 2.4*